# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations_spell.tsv"
pa14_out_filename = "pa14_core_similarity_associations_spell.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression_stats_spell.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression_stats_spell.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 14)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PAO1 id,,,,,,,,,,,,,,
PA2485,PA14_32490,0.390820,9.957665e-195,NaN,NaN,289.535372,1159.336390,1.719518,45.300128,88.641026,194.165203,19397.167941,1.344061e+06,19395.448423
PA3680,PA14_16770,0.354704,2.310475e-158,NaN,NaN,119.056975,85.738349,1.924321,63.193584,102.066035,151.253210,756.763040,7.351064e+03,754.838719
PA3374,PA14_20430,0.375806,4.809487e-179,NaN,NaN,83.060167,100.964905,0.770312,40.502974,59.673651,90.937918,1214.987023,1.019391e+04,1214.216711
PA3123,PA14_23720,0.333443,4.591892e-139,NaN,NaN,274.150213,288.313199,2.815570,84.624468,178.559551,358.328868,2391.170342,8.312450e+04,2388.354773
PA0912,PA14_52465,0.195673,2.603276e-47,NaN,least stable,76.751923,173.179654,0.626660,17.385856,38.800977,81.763985,2930.259548,2.999119e+04,2929.632888


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5348, 14)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PA14 id,,,,,,,,,,,,,,
PA14_58130,PA4480,0.485864,5.785136e-315,mreC,NaN,366.694273,322.204012,25.327191,137.152680,272.534370,482.234632,1820.127275,1.038154e+05,1794.800084
PA14_39860,PA1907,0.289280,1.330534e-103,NaN,NaN,64.577044,127.461242,0.623644,10.275113,21.280266,51.646098,804.399213,1.624637e+04,803.775569
PA14_53800,PA0809,0.272926,5.488988e-92,mntH2,NaN,71.290232,103.376248,2.089569,19.012715,34.526147,86.449821,1381.866084,1.068665e+04,1379.776515
PA14_53010,PA0870,0.360369,9.884890e-164,phhC,NaN,1794.114120,1562.132080,119.787739,619.813449,1359.821622,2523.616169,11430.429988,2.440257e+06,11310.642248
PA14_52280,PA0926,0.328868,4.410338e-135,NaN,NaN,106.347083,62.330320,12.592153,69.317503,93.852104,123.540965,553.295320,3.885069e+03,540.703168


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA1736, PA2003, PA2001, PA4785, PA1999, PA255..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA3454, PA2815, PA2553, PA0879, PA3925, PA044..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA3331, PA1027, PA3426, PA1737, PA2475, PA489..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA2553, PA3366, PA0421, PA3925, PA0202, PA044..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA1736, PA2001, PA3650, PA4785, PA2553, PA380..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA2485,[]
PA3680,[]
PA3374,[KEGG-Pathway-pae00440: Phosphonate and phosph...
PA3123,[]
PA0912,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_32490,[]
PA14_16770,[]
PA14_20430,[KEGG-Pathway-pae00440: Phosphonate and phosph...
PA14_23720,[]
PA14_52465,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 15)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PAO1 id,,,,,,,,,,,,,,,
PA2485,PA14_32490,0.390820,9.957665e-195,NaN,NaN,289.535372,1159.336390,1.719518,45.300128,88.641026,194.165203,19397.167941,1.344061e+06,19395.448423,[]
PA3680,PA14_16770,0.354704,2.310475e-158,NaN,NaN,119.056975,85.738349,1.924321,63.193584,102.066035,151.253210,756.763040,7.351064e+03,754.838719,[]
PA3374,PA14_20430,0.375806,4.809487e-179,NaN,NaN,83.060167,100.964905,0.770312,40.502974,59.673651,90.937918,1214.987023,1.019391e+04,1214.216711,[KEGG-Pathway-pae00440: Phosphonate and phosph...
PA3123,PA14_23720,0.333443,4.591892e-139,NaN,NaN,274.150213,288.313199,2.815570,84.624468,178.559551,358.328868,2391.170342,8.312450e+04,2388.354773,[]
PA0912,PA14_52465,0.195673,2.603276e-47,NaN,least stable,76.751923,173.179654,0.626660,17.385856,38.800977,81.763985,2930.259548,2.999119e+04,2929.632888,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5348, 15)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PA14 id,,,,,,,,,,,,,,,
PA14_58130,PA4480,0.485864,5.785136e-315,mreC,NaN,366.694273,322.204012,25.327191,137.152680,272.534370,482.234632,1820.127275,1.038154e+05,1794.800084,[]
PA14_39860,PA1907,0.289280,1.330534e-103,NaN,NaN,64.577044,127.461242,0.623644,10.275113,21.280266,51.646098,804.399213,1.624637e+04,803.775569,[]
PA14_53800,PA0809,0.272926,5.488988e-92,mntH2,NaN,71.290232,103.376248,2.089569,19.012715,34.526147,86.449821,1381.866084,1.068665e+04,1379.776515,[]
PA14_53010,PA0870,0.360369,9.884890e-164,phhC,NaN,1794.114120,1562.132080,119.787739,619.813449,1359.821622,2523.616169,11430.429988,2.440257e+06,11310.642248,[KEGG-Pathway-pae00360: Phenylalanine metaboli...
PA14_52280,PA0926,0.328868,4.410338e-135,NaN,NaN,106.347083,62.330320,12.592153,69.317503,93.852104,123.540965,553.295320,3.885069e+03,540.703168,[]


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")